In [39]:
import sys
# sys.path.append("C:/Users/ping/MyDrive/py_files/python/py379/")
sys.path.append("py379")
from myUtils import pickle_load, pickle_dump

In [ ]:
import pandas as pd
df1 = pd.read_csv('C:/Users/ping/OneDrive/Documents/jenn_bb_sales/items-2021-02-01-2021-10-02.csv')
df2 = pd.read_csv('C:/Users/ping/OneDrive/Documents/jenn_bb_sales/items-2021-09-06-2022-09-07.csv')
df = pd.concat([df1, df2])
df = df.drop_duplicates()
df.shape

In [ ]:
path_pickle_dump = 'C:/Users/ping/OneDrive/Documents/jenn_bb_sales/'

import sys
sys.path.append("G:/My Drive/py_files/python/perfstat/")
from . import pickle_load, pickle_dump
pickle_dump(df, path_pickle_dump, 'df_items')
df = pickle_load(path_pickle_dump, 'df_items')
df.shape